In [ ]:
#@title Improts
from transformers import AutoModelForObjectDetection, AutoImageProcessor
from PIL import Image
import torch

In [ ]:
#@title Loading the Model and the Dataset
processor = AutoImageProcessor.from_pretrained(
    "justjuu/rtdetr-v2-license-plate-detection"
)
model = AutoModelForObjectDetection.from_pretrained(
    "justjuu/rtdetr-v2-license-plate-detection"
)

In [ ]:
from datasets import load_dataset
ds = load_dataset("justjuu/license-plate-detection")

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import torch

In [ ]:
#@title Evaluation Pipeline
metric = MeanAveragePrecision(
    iou_type="bbox",
    iou_thresholds=[0.5]  # mAP@0.5
)

In [ ]:
def coco_to_xyxy(box):
    x, y, w, h = box
    return [x, y, x + w, y + h]

In [ ]:
from tqdm import tqdm
model.eval()

for sample in tqdm(ds["test"]):
    image = sample["image"]
    gt_boxes = sample["objects"]["bbox"]

    # Convert GT boxes to XYXY
    gt_boxes_xyxy = torch.tensor(
        [coco_to_xyxy(b) for b in gt_boxes],
        dtype=torch.float32
    )

    gt_labels = torch.zeros(len(gt_boxes), dtype=torch.int64)

    # Model inference
    inputs = processor(image, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    preds = processor.post_process_object_detection(
        outputs,
        target_sizes=torch.tensor(
            [(image.height, image.width)]
        ).to(model.device),
        threshold=0.5
    )[0]

    # Format predictions
    pred_dict = {
        "boxes": preds["boxes"].cpu(),
        "scores": preds["scores"].cpu(),
        "labels": preds["labels"].cpu()
    }

    gt_dict = {
        "boxes": gt_boxes_xyxy,
        "labels": gt_labels
    }

    metric.update([pred_dict], [gt_dict])

In [ ]:
results = metric.compute()
results